# Lab Exercise 04

In [1]:
__author__ = "Víctor Vega Sobral"

---

## 1. Local Speech to Text 

In [2]:
import whisper

### 1.1 Loading the Whisper Turbo model

In [3]:
model = whisper.load_model("turbo")

c:\Users\victo\miniconda3\envs\f1_strat_manager\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_loca

### 1.2 Padding to 30 seconds

In [4]:
# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("55_radio_47.mp3")
audio = whisper.pad_or_trim(audio)


### 1.3 Log-Mel spectogram

In [5]:
# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio, n_mels=model.dims.n_mels).to(model.device)
# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

Detected language: en


### 1.4 Decoding the audio

In [6]:
# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)
# print the recognized text
print(result.text)

Rain expected in 4 minutes. What is the rain intensity? 2 to 3.


---

## 2. Local Text to Speech